In [1]:
import os


# # 修改一个已存在的环境变量
# os.environ['PATH'] = '/usr/local/cuda-11.6/bin:' + os.environ['PATH']
# os.environ['LIBRARY_PATH'] = '/usr/local/cuda-11.6/lib64:' + os.environ['LIBRARY_PATH']
# os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.6/lib64:'+ os.environ['LD_LIBRARY_PATH']
# 使用设置好的环境变量
os.environ['CUDA_VISIBLE_DEVICES'] = "7"
print(f"PATH:{os.environ['PATH']}\nCUDA_HOME:{os.environ['CUDA_HOME']}")
dataset_name = "170Kailuan-relu-4"

PATH:/data4/conda_envs/g2/bin:/usr/local/cuda-11.6/bin:/home/beihang/.vscode-server/bin/0ee08df0cf4527e40edc9aa28f4b5bd38bbff2b2/bin/remote-cli:/usr/bin:/data4/conda_envs/g2/bin:/sbin:/usr/local/cuda-10.1/bin:/home/beihang/anaconda3/bin:/home/beihang/anaconda3/condabin:/bin:/usr/bin:/usr/local/bin:/usr/local/cuda-10.1/bin:/home/beihang/anaconda3/bin:/bin:/usr/bin:/bin:/usr/bin:/usr/local/bin:/usr/local/cuda-10.1/bin:/home/beihang/anaconda3/bin:/bin:/usr/bin:/usr/local/cuda/bin:/snap/bin
CUDA_HOME:/usr/local/cuda-11.6


In [2]:
# from aeon.datasets.tsc_data_lists import multivariate_equal_length
import pandas as pd
import numpy as np
from sklearn import preprocessing
import sklearn
from tensorflow import keras
import tensorflow as tf
import numpy as np
import time 
from utils import geng
from utils.utils import create_directory
from utils import logconfig

2024-01-19 07:20:19.853520: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-19 07:20:21.219446: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# multivariate_equal_length

# Data process

## Example of Data

In [4]:
# from aeon.datasets import load_classification
# import numpy as np
# dataset_name = "Libras"
# archive_name="MTS"
# X, Y,meta_data = load_classification(dataset_name,return_metadata=True)
# X_reshaped = np.transpose(X, (0, 2, 1))
# X = X_reshaped
# nb_classes = len(np.unique(Y, axis=0))
# nb_classes,X.shape,Y.shape,np.unique(Y)

## Data Kailuan

In [5]:

archive_name="MTS"
time_series1 = pd.read_csv("./combined_sheets.csv")
time_series2 = pd.read_csv("./combined_sheets2.csv")
meta_data = pd.read_csv("./combinde170_info.csv")
time_series1.shape,time_series2.shape,meta_data.shape 

((140088, 6), (39711, 8), (170, 35))

In [6]:
time_series2 = time_series2.drop(['SystolicPressure','DiastolicPressure'],axis=1)
time_series = pd.concat([time_series1, time_series2], axis=0)
time_series

,Timestamp,Sleepstate,Breath,Heartrate,Movement,Source
0,2021-07-02 16:35:25,4,16,78,2,1
1,2021-07-02 16:36:25,4,16,76,0,1
2,2021-07-02 16:37:25,4,16,74,0,1
3,2021-07-02 16:38:25,4,16,74,0,1
4,2021-07-02 16:39:25,4,16,73,0,1
...,...,...,...,...,...,...
39706,2023-12-22 22:58:00,4,17,86,0,170
39707,2023-12-22 22:59:00,2,17,87,0,170
39708,2023-12-22 23:00:00,4,17,87,0,170
39709,2023-12-22 23:01:00,4,18,88,0,170


In [7]:
# Movement全变成1 二值化
# time_series['Movement'] = np.where(time_series['Movement'] != 0, 1, 0)

## 构造每个source的二维数组

In [8]:
grouped = time_series.groupby('Source')

# 提取特征列
features = [ 'Breath', 'Heartrate']

# # 将每个组转换为二维数组并堆叠
# three_dim_array = np.array([group[features].values for _, group in grouped])

# three_dim_array.shape # 显示三维数组的形状

## Movement 处理
- Movement 取消： 只有fold3有提升 其他4个fold下降
- Movement 全部+1 不行 都有下降


In [9]:
# time_groupby = time_series.groupby('Source')

In [10]:
# 查看movement的分布
# movement_stac=[{}]
# for i in range(1,136):
#     print(i)
#     keys, vals = np.unique(time_groupby.get_group(int(i))["Movement"].values,return_counts=True)
#     result_dict = dict(zip(keys, vals))
#     print(result_dict)
#     movement_stac.append(result_dict)

# # movement_stac

## 填充 截断 时间序列

## 填充/截断 构造数据集(去掉过少的)

In [11]:
# trial block 


In [12]:
# 重新计算每个样例的时间步数（考虑到了数据过滤）
filtered_time_steps_per_source = time_series.groupby('Source').size()

# 重新筛选出有效的 Source 过滤掉过少的
valid_sources = filtered_time_steps_per_source[filtered_time_steps_per_source > 500].index
obsolete_sources = filtered_time_steps_per_source[filtered_time_steps_per_source <= 500].index
print("valid sources:", valid_sources, "\nobsolete sources:", obsolete_sources, "\n len of obsoletes:",filtered_time_steps_per_source[obsolete_sources])
# 重新获取有效的数据
valid_filtered_time_series = time_series[time_series['Source'].isin(valid_sources)]

# 重新分组
valid_filtered_grouped = valid_filtered_time_series.groupby('Source')

# 重新计算平均时间步数（此时应该没有 NaN 值）
average_time_steps = int(valid_filtered_grouped.size().mean())

print(f"\n average_time_steps:{average_time_steps}")

# 初始化新的三维数组（考虑到了有效的样例数）
valid_sources_count = len(valid_sources)
three_dim_array_valid = np.zeros((valid_sources_count, average_time_steps, len(features)))

# 填充新的三维数组
for i, (source, group) in enumerate(valid_filtered_grouped):
    data = group[features].values
    current_steps = data.shape[0]
    if current_steps < average_time_steps:
        fill_values = {
            'Breath': group['Breath'].mean(),
            'Heartrate': group['Heartrate'].mean(),
        }
        fill_array = np.array([[fill_values[feature] for feature in features]] * (average_time_steps - current_steps))
        full_data = np.vstack(( fill_array,data))
    else:
        # 调整为截取较后面的元素
        full_data = data[current_steps-average_time_steps:, :]
    # print(f"source:{source},current_steps:{current_steps}, full_data shape:{full_data.shape}")
    three_dim_array_valid[i, :, :] = full_data


final_data = three_dim_array_valid

# 进行数据标准化
final_data = geng.standard_scaler_total(final_data)

final_data.shape

valid sources: Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170],
      dtype='int64', name='Source', length=164) 
obsolete sources: Index([21, 26, 74, 75, 107, 130], dtype='int64', name='Source') 
 len of obsoletes: Source
21     148
26     251
74     193
75     150
107    415
130    188
dtype: int64

 average_time_steps:1088


(164, 1088, 2)

### 标签构造
- 意识状态
- GCS

In [13]:
valid_sources

Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170],
      dtype='int64', name='Source', length=164)

In [14]:
labels = meta_data["意识状态"].values

valid_sources = valid_sources-1
labels = [labels[i] for i in range(len(labels)) if i in valid_sources]
# labels = [1 if x == 2 else x for x in labels]
nb_classes = len(np.unique(labels, axis=0))
labels = np.array(labels)
labels, labels.shape

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
        2, 2, 2, 1, 1, 1, 1, 1, 2, 1]),
 (164,))

## (填充/截断)构造数据集 原本版 

In [ ]:

# average_time_steps = int(grouped.apply(lambda x: x.shape[0]).mean())

# # 初始化三维数组
# features = ['Sleepstate', 'Breath', 'Heartrate', 'Movement']
# three_dim_array = np.zeros((135, average_time_steps, len(features)))

# # 填充三维数组
# for source, group in grouped:
#     data = group[features].values
#     current_steps = data.shape[0]

#     if current_steps < average_time_steps:
#         fill_values = {
#             'Breath': group['Breath'].mean(),
#             'Heartrate': group['Heartrate'].mean(),
#             'Sleepstate': group['Sleepstate'].median(),
#             'Movement': 0
#         }
#         fill_array = np.array([[fill_values[feature] for feature in features]] * (average_time_steps - current_steps))
#         full_data = np.vstack((data, fill_array))
#     else:
#         full_data = data[:average_time_steps, :]

#     three_dim_array[source-1, :, :] = full_data

# three_dim_array.shape  # 显示三维数组的形状


### 标签构造
- 意识状态
- GCS

In [ ]:
# labels = meta_data["意识状态"].values
# nb_classes = len(np.unique(labels, axis=0))
# labels,nb_classes

In [ ]:
def data_final_process(x,y):
    nb_classes = len(np.unique(y, axis=0))
    # transform the labels from integers to one hot vectors
    enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
    enc.fit(y.reshape(-1, 1))
    y = enc.transform(y.reshape(-1, 1)).toarray()

    # # save orignal y because later we will use binary
    # y_true = np.argmax(y_test, axis=1)
    print(f'x.shape: {x.shape}, y.shape: {y.shape}\n')
    if len(x.shape) == 2:  # if univariate
        # add a dimension to make it multivariate with one dimension 
        x = x.reshape((x.shape[0], x.shape[1], 1))

    input_shape = x.shape[1:]
    print(f'input_shape: {input_shape}\n')
    
    return x,y,input_shape

In [ ]:
# classifier_name="fcn"
# root_dir="/data4/gsprivate/dl-4-tsc"
# output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + '' + '/' + \
#                     dataset_name + '/'
                    
# if create_directory(output_directory) is None:
#     print("Creating directory:{} None".format(output_directory))

# x,y,input_shape=data_final_process(final_data,labels)
# fcn_classifier = geng.create_classifier(classifier_name, input_shape, nb_classes, output_directory)
# print(f"Classifier x.shape={x.shape} ,y.shape={y.shape}, input_shape={input_shape}, nb_classes={nb_classes}")

In [ ]:
# classifier_name="cnn"
# root_dir="/data4/gsprivate/dl-4-tsc"
# output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + '' + '/' + \
#                     dataset_name + '/'
                    
# if create_directory(output_directory) is None:
#     print("Creating directory:{} None".format(output_directory))

# x,y,input_shape=data_final_process(final_data,labels)
# cnn_classifier = geng.create_classifier(classifier_name, input_shape, nb_classes, output_directory)
# print(f"Classifier x.shape={x.shape} ,y.shape={y.shape}, input_shape={input_shape}, nb_classes={nb_classes}")

In [ ]:
classifier_name="mcnn"
root_dir="/data4/gsprivate/dl-4-tsc"
output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + '' + '/' + \
                    dataset_name + '/'
                    
if create_directory(output_directory) is None:
    print("Creating directory:{} None".format(output_directory))

x,y,input_shape=data_final_process(final_data,labels)
mcnn_classifier = geng.create_classifier(classifier_name, input_shape, nb_classes, output_directory)
print(f"Classifier x.shape={x.shape} ,y.shape={y.shape}, input_shape={input_shape}, nb_classes={nb_classes}")

In [ ]:
# %%capture cap --no-stderr
# print("这将写入到文件中")
logconfig.setup_logging(dir=output_directory)
geng.fit_splits_for_mcnn(mcnn_classifier,x,y,epochs=250)
print("DONE")

In [ ]:
# geng.fit_splits(cnn_classifier,x,y,batch_size=8,epochs=200)
# print('DONE')

In [ ]:
# import torch 
import tensorflow as tf

# print(torch.version.cuda)·
# print(torch.backends.cudnn.version())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
# import os

# # Retrieving all environment variables
# env_vars = os.environ

# # Displaying the environment variables
# env_vars_dict = {key: env_vars[key] for key in env_vars}
# env_vars_dict['LD_LIBRARY_PATH']